In [13]:
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [14]:
path = 'dataset'

def Data(path):
    train_img = []
    train_label = []
    test_img = []
    test_label = []
    img_path = os.path.join(path, 'img')
    csv_file = open(os.path.join(path, 'label.csv'))
    reader = list(csv.reader(csv_file))
    reader = reader[1:]
    for i in range(3200):
        if int(reader[i][0])<10:
            img = cv2.imread(os.path.join(img_path, '000'+reader[i][0]+'.jpg'))
        elif int(reader[i][0])<100:
            img = cv2.imread(os.path.join(img_path, '00'+reader[i][0]+'.jpg'))
        elif int(reader[i][0])<1000:
            img = cv2.imread(os.path.join(img_path, '0'+reader[i][0]+'.jpg'))
        else:
            img = cv2.imread(os.path.join(img_path, reader[i][0]+'.jpg'))
        img = cv2.resize(img, dsize=(80, 80))
        train_img.append(img)
        train_label.append(reader[i][1]=='real')
        if (i+1)%200==0:
            print('{}/3200 train data preprocessing finished'.format(i+1))
    print()

    for i in range(3200, 4000):
        img = cv2.imread(os.path.join(img_path, reader[i][0]+'.jpg'))
        img = cv2.resize(img, dsize=(80, 80))
        test_img.append(img)
        test_label.append(reader[i][1]=='real')
        if (i-3199)%200==0:
            print('{}/400 test data preprocessing finished'.format(i-3199))

    train_img = np.array(train_img).reshape(-1, 80, 80, 3)
    train_img = train_img/255.0
    train_label = np.array(train_label)

    test_img = np.array(test_img).reshape(-1, 80, 80, 3)
    test_img = test_img/255.0
    test_label = np.array(test_label)
    return train_img, train_label, test_img, test_label


In [15]:
train_X, train_Y, test_X, test_Y = Data(path)

200/3200 train data preprocessing finished
400/3200 train data preprocessing finished
600/3200 train data preprocessing finished
800/3200 train data preprocessing finished
1000/3200 train data preprocessing finished
1200/3200 train data preprocessing finished
1400/3200 train data preprocessing finished
1600/3200 train data preprocessing finished
1800/3200 train data preprocessing finished
2000/3200 train data preprocessing finished
2200/3200 train data preprocessing finished
2400/3200 train data preprocessing finished
2600/3200 train data preprocessing finished
2800/3200 train data preprocessing finished
3000/3200 train data preprocessing finished
3200/3200 train data preprocessing finished

200/400 test data preprocessing finished
400/400 test data preprocessing finished
600/400 test data preprocessing finished
800/400 test data preprocessing finished


In [16]:
def Create_CNN():
    model=Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(80, 80, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

model=Create_CNN()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 78, 78, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 39, 39, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 37, 37, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 18, 18, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20736)             0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [17]:
model.fit(train_X, train_Y, epochs=10, batch_size=32)

Epoch 1/10
100/100 [==============================] - 17s 161ms/step - loss: 0.7119 - accuracy: 0.4812
Epoch 2/10
100/100 [==============================] - 17s 168ms/step - loss: 0.6928 - accuracy: 0.5191
Epoch 3/10
100/100 [==============================] - 16s 164ms/step - loss: 0.6832 - accuracy: 0.5644
Epoch 4/10
100/100 [==============================] - 16s 162ms/step - loss: 0.6504 - accuracy: 0.6075
Epoch 5/10
100/100 [==============================] - 16s 160ms/step - loss: 0.5964 - accuracy: 0.6597
Epoch 6/10
100/100 [==============================] - 16s 161ms/step - loss: 0.4916 - accuracy: 0.7356
Epoch 7/10
100/100 [==============================] - 16s 161ms/step - loss: 0.3449 - accuracy: 0.8378
Epoch 8/10
100/100 [==============================] - 16s 162ms/step - loss: 0.2130 - accuracy: 0.9191
Epoch 9/10
100/100 [==============================] - 16s 163ms/step - loss: 0.1394 - accuracy: 0.9544
Epoch 10/10
100/100 [==============================] - 16s 162ms/step - l

In [18]:
predictions=model.predict(test_X)
predict_val=[int(round(p[0])) for p in predictions]
ac = 0
for i in range(800):
    if test_Y[i]==predict_val[i]:
        ac += 1
print("Accuracy: {}".format(ac/800))

25/25 [==============================] - 1s 29ms/step
Accuracy: 0.495
